In [1]:
import os
import json
import re
from lxml import html
import pandas as pd


def clean_page(page):
    HTML_TAGS_TO_REMOVE = [
        r'<\/?p(?: \w+="[\w.:%&;@#$,\?]+"){0,7}>',
        r'<\/?h\d>',
        r'<\/?b>',
        r'<\/?i>',
        r'<\/?u>',
        r'<\/?ul>',
        r'<\/?li>',
        r'<br ?\/?>',
        r'&nbsp;'
    ]

    regex_pattern = r'(?:{})'.format('|'.join(HTML_TAGS_TO_REMOVE))
    page_cleaned = re.sub(regex_pattern, ' ', page)
    page_cleaned = re.sub(r'[?.!]', '.', page_cleaned)
    page_cleaned = page_cleaned.lower() \
        .replace('\xa0', ' ')\
        .replace(r'</li>', '.')

    return page_cleaned


def parse_descr_from_page(page):
    KEYWORDS = [
        'requirements',
        'description',
        'role',
        'requirements',
        'education',
        'qualifications',
        'job title',
        'what you',
        'who we',
        'position summary',
        'background'
    ]

    contains_predicate = ' or '.join('contains(text(),"{}")'.format(kw) for kw in KEYWORDS)
    descr_xpath = '//div[{pred}]//text()'.format(pred=contains_predicate)

    tree = html.fromstring(page)
    job_descr = ''.join(tree.xpath(descr_xpath))

    return job_descr

In [3]:
SCRAPE_DIR = r'C:\Users\rmdelgad\Documents\repos\ryans_projects\datascientist\data\scraped\20171226'
scrape_files = os.listdir(SCRAPE_DIR)

for sf in scrape_files:
    print('Processing {}'.format(sf))
    with open(os.path.join(SCRAPE_DIR, sf), 'r') as f:
        scrape_dict = json.load(f)

    page_raw = scrape_dict['body']
    page_cleaned = clean_page(page_raw)
    job_descr = parse_descr_from_page(page_cleaned)

    cleaned_dict = {
        'description': job_descr,
        'company': scrape_dict['company'],
        'title': scrape_dict['title'],
        'url': scrape_dict['job_url']
    }

    CLEANED_DIR = r'C:\Users\rmdelgad\Documents\repos\ryans_projects\datascientist\data\cleaned\20171226'
    with open(os.path.join(CLEANED_DIR, sf), 'w') as f:
        json.dump(cleaned_dict, f)

Processing 23andMeInc_DataScientist_20171226144142279635.json
Processing 23andMeInc_DataScientist_20171226144322998282.json
Processing 3M_ProcessDataScientistWoodburyMN_20171226144057971386.json
Processing 6sense_DataScientist_20171226144138565495.json
Processing 7Park_DataScientist_20171226144037477607.json
Processing 8451_ResearchScientist_20171226144153642151.json
Processing A9com_DataSciencesEngineerMachineLearningSystems_20171226144321215450.json
Processing AbbVie_DataScientistICheminformatics_20171226144206513147.json
Processing ABB_MachinelearningandDataAnalyticsResearcher_20171226144209142362.json
Processing AccoliteInc_SeniorDataScientist_20171226144028759635.json
Processing Acorns_DataScientistTargeting_20171226144354521345.json
Processing Adobe_DataScientistAIML_20171226144057079183.json
Processing Adobe_MachineLearningScientist_20171226144353464994.json
Processing AdRoll_StaffDataScientistIncrementaility_20171226144414298957.json
Processing AdTheorent_DataScientist_20171226

XMLSyntaxError: line 252: b'Opening and ending tag mismatch: a and i' (<string>)